In [ ]:
from deep_translator import GoogleTranslator
import google.generativeai as genai
import deepl
from data_class import Data
from typing import Optional
import time
import random

from configuration.config import API_KEY_DEEPL, API_KEY_GEMINI

c:\Users\casabik\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_manager = Data(db_path="data/data.db")

In [ ]:
# GoogleTranslator

def translate_column_to_english(data_manager: Data, table_name: str = 'train',
                              source_column: str = 'task', 
                              target_column: str = 'translated_google'):
    data_manager.prepare_translation_column(table_name, target_column)
    
    rows = data_manager.get_data_for_translation(table_name, source_column)
    
    print(f"Начинаем перевод {len(rows)} записей...")
    
    for i, (row_id, text) in enumerate(rows, 1):
        if text:
            translated_text = translate_text(text, 'en')
            
            data_manager.save_translated_text(table_name, target_column, 
                                            row_id, translated_text)
            
    
            if i % 5 == 0:  
                time.sleep(random.uniform(0.5, 1.5))
            
            if i % 100 == 0: 
                data_manager.commit_changes()
                print(f"Переведено {i}/{len(rows)} записей...")
    
    data_manager.commit_changes()
    print(f"Перевод завершен! Всего переведено {len(rows)} записей.")


def translate_text(text: str, target_language: str = 'en') -> str:
    """Переводит текст на указанный язык с несколькими попытками и задержками"""
    if not text or not isinstance(text, str):
        return text
    
    max_attempts = 3
    
    for attempt in range(max_attempts):
        try:
            translator = GoogleTranslator(source='auto', target=target_language)
            translated = translator.translate(text)
            return translated
            
        except Exception as e:
            print(f"Попытка {attempt + 1}/{max_attempts} не удалась для '{text[:50]}...': {e}")
            
            if attempt < max_attempts - 1:
                delay = random.uniform(1, 5)
                print(f"Ждем {delay:.1f} секунд перед следующей попыткой...")
                time.sleep(delay)
            else:
                print(f"Все попытки исчерпаны для текста: '{text[:50]}...'")
    
    return text
    

#translate_column_to_english(data_manager)
#translate_column_to_english(data_manager, table_name='test')

Начинаем перевод 1000 записей...
Переведено 100/1000 записей...
Переведено 200/1000 записей...
Переведено 300/1000 записей...
Переведено 400/1000 записей...
Переведено 500/1000 записей...
Переведено 600/1000 записей...
Переведено 700/1000 записей...
Переведено 800/1000 записей...
Переведено 900/1000 записей...
Переведено 1000/1000 записей...
Перевод завершен! Всего переведено 1000 записей.
Начинаем перевод 200 записей...
Переведено 100/200 записей...
Переведено 200/200 записей...
Перевод завершен! Всего переведено 200 записей.


In [6]:
# DEEPL
def translate_text_deepl(text: str, api_key: str, target_language: str = 'EN-US') -> str:
    if not text or not isinstance(text, str):
        return text
    
    max_attempts = 3
    
    for attempt in range(max_attempts):
        try:
            translator = deepl.Translator(api_key)
            result = translator.translate_text(text, target_lang=target_language)
            return str(result)
            
        except deepl.exceptions.AuthorizationException:
            print(f"Ошибка авторизации DeepL API. Проверьте ключ API.")
            return text
            
        except deepl.exceptions.QuotaExceededException:
            print(f"Превышена квота DeepL API.")
            return text
            
        except Exception as e:
            print(e)
            print(f"Попытка {attempt + 1}/{max_attempts} не удалась для DeepL '{text[:50]}...': {e}")
            
            if attempt < max_attempts - 1:
                delay = random.uniform(1, 3)
                print(f"Ждем {delay:.1f} секунд перед следующей попыткой...")
                time.sleep(delay)
            else:
                print(f"Все попытки DeepL исчерпаны для текста: '{text[:50]}...'")
    
    return text


def translate_column_to_english_deepl(data_manager: Data, api_key: str,
                                    table_name: str = 'train',
                                    source_column: str = 'task', 
                                    target_column: str = 'translated_deepl'):

    data_manager.prepare_translation_column(table_name, target_column)
    

    rows = data_manager.get_data_for_translation(table_name, source_column, target_column)
    
    print(f"Начинаем перевод DeepL {len(rows)} записей...")
    
    for i, (row_id, text, language) in enumerate(rows, 1):
        if text:
            translated_text = translate_text_deepl(text, api_key, 'EN-US')
            
            data_manager.save_translated_text(table_name, target_column, 
                                            row_id, translated_text)
            
            if i % 5 == 0:
                time.sleep(random.uniform(0.3, 1.0))
            
            if i % 100 == 0:  
                data_manager.commit_changes()
                print(f"Переведено DeepL {i}/{len(rows)} записей...")
    
    data_manager.commit_changes()
    print(f"Перевод DeepL завершен! Всего переведено {len(rows)} записей.")


#translate_column_to_english_deepl(data_manager, API_KEY_DEEPL)
#translate_column_to_english_deepl(data_manager, API_KEY_DEEPL, table_name='test')
    

In [ ]:
#GEMINI

def translate_text_gemini(text: str, api_key: str, language: Optional[str] = None) -> str:
    """Переводит текст на русский язык с помощью Gemini API, с fallback на резервные модели"""
    if not text or not isinstance(text, str):
        return text

    if language == 'ru':
        return text

    genai.configure(api_key=api_key)

    models = ["gemini-2.5-flash", "gemini-2.5-flash-lite", "gemini-2.0-flash"]
    prompt = r"""
Ты — профессиональный переводчик математических задач. Твоя задача — перевести любой входной текст на русский язык, при этом:
1. Во всём тексте сохранить в точности все математические выражения, символы, переменные, формулы и LaTeX-блоки (например, \[...\], \(...\), $...$, окружения equation, align и т. п.), не изменяя их.
2. Не переводить имена переменных, констант, функции, математические операторы и символы («x», «p_i(x)», «+», «=» и т. д.).
3. Сохранить исходную пунктуацию и форматирование (переносы строк, списки, нумерацию).
4. Возвращать только финальный переведённый текст на русском — без пояснений, комментариев, заголовков, метаданных, JSON-обёрток и т. п.

Переведи следующий текст:
{input_text}
    """

    for model_name in models:
        model = genai.GenerativeModel(model_name)
        for attempt in range(3): 
            try:
                response = model.generate_content(prompt.format(input_text=text))
                return response.text.strip()
            except Exception as e:
                error_str = str(e)
                if "429" in error_str or "quota" in error_str.lower():
                    wait_time = 60
                    print(f"[{attempt+1}/3] Превышен лимит или квота для {model_name}. Ждём {wait_time} секунд...")
                    time.sleep(wait_time)
                else:
                    print(f"Ошибка с моделью {model_name}: {error_str[:100]}...")
                    break 

        print(f"Переход на следующую модель после {model_name}...")

    print("Все модели недоступны. Возвращаю исходный текст.")
    return text 


def translate_column_to_russian_gemini(data_manager: Data, api_key: str,
                                       table_name: str = 'train',
                                       source_column: str = 'task', 
                                       target_column: str = 'translated_gemini'):
    """Координирует процесс перевода столбца на русский с помощью Gemini"""
    data_manager.prepare_translation_column(table_name, target_column)
    rows = data_manager.get_data_for_translation(table_name, source_column, target_column)

    print(f"Начинаем перевод Gemini {len(rows)} записей...")

    for i, (row_id, text, language) in enumerate(rows, 1):
        if text:
            translated_text = translate_text_gemini(text, api_key, language)

            data_manager.save_translated_text(table_name, target_column, row_id, translated_text)

            time.sleep(4)

            if i % 50 == 0:  
                data_manager.commit_changes()
                print(f"Переведено Gemini {i}/{len(rows)} записей...")

    data_manager.commit_changes()
    print(f"Перевод Gemini завершен! Всего переведено {len(rows)} записей.")



print("Начинаем перевод Gemini на русский...")
translate_column_to_russian_gemini(data_manager, API_KEY_GEMINI)

Начинаем перевод Gemini на русский...
Начинаем перевод Gemini 150 записей...
Переведено Gemini 50/150 записей...
Переведено Gemini 100/150 записей...
Переведено Gemini 150/150 записей...
Перевод Gemini завершен! Всего переведено 150 записей.
